In [1]:
from funcs.notebook import *

from altaipony.lcio import from_mast
from funcs.detrend import custom_detrending, estimate_detrended_noise
from funcs.phaseanalysis import get_flare_phases

from matplotlib.ticker import FormatStrFormatter

path = "/home/ekaterina/Documents/002_writing/aumic-flaring-spi-draft/figures"


rotper =  4.862 # martioli
orbper =  8.463 # exoplanet.eu

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
aumic1 = pd.read_csv("../results/2021_02_18_AUMic_flares_1.csv")
aumic1 = aumic1[(aumic1.final==1) & (aumic1["real?"]==1)]
aumic27 = pd.read_csv("../results/2021_02_11_AUMic_flares_27.csv")
aumic27 = aumic27[(aumic27.final==1) & (aumic27["real?"]==1)]
aumic_ = pd.concat([aumic1,aumic27])


aumic_.head()

NumExpr defaulting to 8 threads.


,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dur,total_n_valid_data_points,ID,qcs,mission,tstamp,dur_detrend,lc_n,real?,note,final
6,1509,1519,71983,71993,1327.437532,1327.451421,0.661332,0.038413,0.001381,0.013889,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
7,1532,1539,72006,72013,1327.469477,1327.479199,0.659758,0.044819,0.001441,0.009722,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
8,1818,1827,72313,72322,1327.895864,1327.908364,1.669144,0.039894,0.003435,0.012500,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
9,1997,2004,72492,72499,1328.144475,1328.154197,0.923981,0.045494,0.002392,0.009722,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
11,2254,2278,72751,72775,1328.504196,1328.537529,14.728960,0.035002,0.025630,0.033333,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0


In [5]:
aumic1.sort_values(by="ed_rec", ascending=True).iloc[:aumic1.shape[0]//2 +1]

,istart,istop,cstart,cstop,tstart,tstop,ed_rec,ed_rec_err,ampl_rec,dur,total_n_valid_data_points,ID,qcs,mission,tstamp,dur_detrend,lc_n,real?,note,final
47,7889,7892,78603,78606,1336.631911,1336.636078,0.174104,0.026558,0.000550,0.004167,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
62,10628,10631,82198,82201,1341.624874,1341.629041,0.254045,0.031959,0.000798,0.004167,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
64,11311,11314,82903,82906,1342.604015,1342.608182,0.268232,0.029831,0.000972,0.004167,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
12,2304,2307,72801,72804,1328.573640,1328.577806,0.302502,0.030096,0.001242,0.004167,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
61,10506,10510,82076,82080,1341.455434,1341.460990,0.309265,0.040267,0.000957,0.005555,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
73,13220,13223,84839,84842,1345.292829,1345.296996,0.326147,0.026473,0.001072,0.004167,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,has a secondary shortly after decay,1.0
52,8592,8596,79318,79322,1337.624955,1337.630510,0.339966,0.032661,0.001340,0.005555,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
59,9903,9907,81469,81473,1340.612400,1340.617955,0.342896,0.030011,0.001357,0.005555,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
46,7824,7827,78537,78540,1336.540245,1336.544412,0.349228,0.033081,0.001168,0.004167,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0
36,6326,6330,76990,76994,1334.391662,1334.397217,0.352307,0.035775,0.000864,0.005555,18094,AU Mic,1,TESS,18_02_2021_14_40_23,8.047532,0,1.0,NaN,1.0


In [ ]:
0.93

In [35]:
aumic_ = get_flare_phases(aumic_, "Orbit")
aumic_.columns

Index(['istart', 'istop', 'cstart', 'cstop', 'tstart', 'tstop', 'ed_rec',
       'ed_rec_err', 'ampl_rec', 'dur', 'total_n_valid_data_points', 'ID',
       'qcs', 'mission', 'tstamp', 'dur_detrend', 'lc_n', 'real?', 'note',
       'final', 'phases'],
      dtype='object')

In [62]:
restab = aumic_[['qcs', 'tstart', 'tstop', 'phases',
                 'ed_rec', 'ed_rec_err', 'ampl_rec']].round(4).sort_values(by="phases")

restab[r"$ED$ [s]"] = restab.apply(lambda x: fr"${x.ed_rec:.2f} \pm {x.ed_rec_err:.2f}$", axis=1)
restab.ampl_rec = restab.ampl_rec.round(3)
restab.phases = restab.phases.round(3)

In [63]:
del restab["ed_rec"]
del restab["ed_rec_err"]

In [64]:
restab = restab.rename(index=str, columns={"qcs":"Sec.",
                                           "tstart":r"$t_s$ [BTJD]",
                                           "tstop":r"$t_f$ [BTJD]",
                                           "phases":"orb. phase",
                                           "ampl_rec":r"$a$",
                                            })
restab

,Sec.,$t_s$ [BTJD],$t_f$ [BTJD],orb. phase,$a$,$ED$ [s]
167,27,2058.2378,2058.2526,0.001,0.007,$3.48 \pm 0.04$
168,27,2058.2584,2058.2593,0.004,0.002,$0.10 \pm 0.01$
41,27,2041.3551,2041.3572,0.006,0.002,$0.23 \pm 0.02$
20,1,1330.4514,1330.4709,0.007,0.003,$1.65 \pm 0.03$
42,27,2041.3690,2041.3734,0.008,0.003,$0.70 \pm 0.03$
...,...,...,...,...,...,...
209,27,2058.0725,2058.0875,0.982,0.002,$0.66 \pm 0.03$
40,27,2041.2065,2041.2100,0.989,0.002,$0.40 \pm 0.02$
87,1,1347.2692,1347.2803,0.994,0.001,$0.94 \pm 0.07$
92,27,2049.7375,2049.7430,0.997,0.007,$1.54 \pm 0.03$


In [65]:
restab = restab[["Sector",r"$t_s$ [BTJD]", r"$t_f$ [BTJD]", "orb. phase", r"$a$", r"$ED$ [s]"]]

KeyError: "['Sector'] not in index"

In [66]:
stri = restab.head(10).to_latex(index=False,escape=False, column_format=f"l|ccccc")
stri = stri.replace("\\toprule","\hline")
stri = stri.replace("\\midrule","\hline")
stri = stri.replace("{}","\hline")
stri = stri.replace("\\bottomrule","\hline\n" )
print(stri)

\begin{tabular}{l|ccccc}
\hline
 Sec. &  $t_s$ [BTJD] &  $t_f$ [BTJD] &  orb. phase &    $a$ &         $ED$ [s] \\
\hline
   27 &     2058.2378 &     2058.2526 &       0.001 &  0.007 &  $3.48 \pm 0.04$ \\
   27 &     2058.2584 &     2058.2593 &       0.004 &  0.002 &  $0.10 \pm 0.01$ \\
   27 &     2041.3551 &     2041.3572 &       0.006 &  0.002 &  $0.23 \pm 0.02$ \\
    1 &     1330.4514 &     1330.4709 &       0.007 &  0.003 &  $1.65 \pm 0.03$ \\
   27 &     2041.3690 &     2041.3734 &       0.008 &  0.003 &  $0.70 \pm 0.03$ \\
   27 &     2041.6186 &     2041.6202 &       0.038 &  0.002 &  $0.15 \pm 0.02$ \\
   27 &     2050.1252 &     2050.1259 &       0.043 &  0.003 &  $0.10 \pm 0.01$ \\
   27 &     2050.1724 &     2050.1731 &       0.048 &  0.001 &  $0.07 \pm 0.01$ \\
    1 &     1330.8028 &     1330.8250 &       0.049 &  0.002 &  $1.72 \pm 0.06$ \\
   27 &     2058.7093 &     2058.7148 &       0.057 &  0.004 &  $0.74 \pm 0.03$ \\
\hline

\end{tabular}



<>:2: DeprecationWarning: invalid escape sequence \h
<>:3: DeprecationWarning: invalid escape sequence \h
<>:4: DeprecationWarning: invalid escape sequence \h
<>:5: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:3: DeprecationWarning: invalid escape sequence \h
<>:4: DeprecationWarning: invalid escape sequence \h
<>:5: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:3: DeprecationWarning: invalid escape sequence \h
<>:4: DeprecationWarning: invalid escape sequence \h
<>:5: DeprecationWarning: invalid escape sequence \h
<ipython-input-66-5876a08b8638>:2: DeprecationWarning: invalid escape sequence \h
  stri = stri.replace("\\toprule","\hline")
<ipython-input-66-5876a08b8638>:3: DeprecationWarning: invalid escape sequence \h
  stri = stri.replace("\\midrule","\hline")
<ipython-input-66-5876a08b8638>:4: DeprecationWarning: invalid escape sequence \h
  stri = stri.replace("{}","\h

In [67]:
with open("/home/ekaterina/Documents/002_writing/aumic-flaring-spi-draft/tables/flares.tex", "w") as f:
    f.write(stri)

In [16]:
print(f"Sector 1: {restab[restab.qcs==1].shape[0]} confirmed flares")
print(f"Sector 27: {restab[restab.qcs==27].shape[0]} confirmed flares")

Sector 1: 75 confirmed flares
Sector 27: 114 confirmed flares


In [50]:
1e-3 * 24 * 60 * 60

86.39999999999999

In [68]:
15/189

0.07936507936507936

In [69]:
15/94

0.1595744680851064

In [78]:
aumic_.ed_rec.sort_values(ascending=True).iloc[94] * 2.3297917e31

1.436955491054707e+31

In [79]:
aumic_.ed_rec.sort_values(ascending=True).iloc[-1] * 2.3297917e31

2.3090671314689408e+33

In [6]:
index = >

SyntaxError: invalid syntax (<ipython-input-6-62f11cbecf57>, line 1)